<a href="https://colab.research.google.com/github/gae-ddong/mlteamproject/blob/main/final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
cd /content/drive/MyDrive/data/

/content/drive/.shortcut-targets-by-id/1G6SgIRKCYt5sUkkaCAiAmdP0_hZA-rFs/data


In [3]:
ls

blurr_cut_png/   Final_data/  images_cut2_png/  images_jpg/  incipient/  no/
cut_resize_png/  first_data/  images_gray_jpg/  images_png/  mature/     overripe/


In [4]:
pip install optuna

In [5]:
pip install tensorflow-addons

In [6]:
pip install vit-keras

In [7]:
import tensorflow as tf
import os
import pickle
import optuna
import pandas as pd
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
from optuna.visualization import plot_optimization_history
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import confusion_matrix, classification_report
import tensorflow_addons as tfa
from vit_keras import vit
from vit_keras import visualize


gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)


/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


1 Physical GPUs, 1 Logical GPUs


In [8]:
images_final_data_path = '/content/drive/My Drive/data/Final_data/'

In [9]:
os.chdir(images_final_data_path)
!ls

grad_cam.jpg  incipient  mature  no  overripe


In [10]:
image_path = images_final_data_path

Pickle load to variable

In [11]:
os.chdir(os.path.join(image_path, 'overripe'))
with open('zero_centering.pkl', 'rb') as f:
     overripe = pickle.load(f)
     print('overripe : ', len(overripe))
os.chdir(os.path.join(image_path, 'no'))
with open('zero_centering.pkl', 'rb') as f:
     no = pickle.load(f)
     print('no : ', len(no))

os.chdir(os.path.join(image_path, 'mature'))
with open('zero_centering.pkl', 'rb') as f:
     mature = pickle.load(f)
     print('mature : ', len(mature))

os.chdir(os.path.join(image_path, 'incipient'))
with open('zero_centering.pkl', 'rb') as f:
     incipient = pickle.load(f)
     print('incipient : ', len(incipient))

overripe :  2000
no :  2000
mature :  2000
incipient :  2000


Dictionary to numpy array

In [12]:
# overripe의 모든 사진에 대해 반복
overripe_data_list = []

for filename, data in overripe.items():
    zero_centering_value = data.get('zero_centering')
    overripe_data_list.append(np.array(zero_centering_value))

# 리스트를 NumPy 배열로 변환
overripe_data = np.array(overripe_data_list)

# 확인을 위해 배열의 크기 출력
print("overripe_data shape:", overripe_data.shape)
no_data_list = []

for filename, data in no.items():
    zero_centering_value = data.get('zero_centering')
    no_data_list.append(np.array(zero_centering_value))

# 리스트를 NumPy 배열로 변환
no_data = np.array(no_data_list)

# 확인을 위해 배열의 크기 출력
print("no_data shape:", no_data.shape)

mature_data_list = []

for filename, data in mature.items():
    zero_centering_value = data.get('zero_centering')
    mature_data_list.append(np.array(zero_centering_value))

# 리스트를 NumPy 배열로 변환
mature_data = np.array(mature_data_list)

# 확인을 위해 배열의 크기 출력
print("mature_data shape:", mature_data.shape)

incipient_data_list = []

for filename, data in incipient.items():
    zero_centering_value = data.get('zero_centering')
    incipient_data_list.append(np.array(zero_centering_value))

# 리스트를 NumPy 배열로 변환
incipient_data = np.array(incipient_data_list)

# 확인을 위해 배열의 크기 출력
print("incipient_data shape:", incipient_data.shape)

overripe_data shape: (2000, 224, 224, 3)
no_data shape: (2000, 224, 224, 3)
mature_data shape: (2000, 224, 224, 3)
incipient_data shape: (2000, 224, 224, 3)


Train Val Test split (0.75:0.15:0.15)

In [13]:
# 데이터를 train/validation/test로 나누기
X = np.concatenate((overripe_data, no_data, mature_data, incipient_data), axis=0)
y = np.concatenate((np.zeros(overripe_data.shape[0]), np.ones(no_data.shape[0]),
                    2*np.ones(mature_data.shape[0]), 3*np.ones(incipient_data.shape[0])))

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

(5600, 224, 224, 3)
(1200, 224, 224, 3)
(1200, 224, 224, 3)


y one-hot

In [14]:
y_train_one_hot = to_categorical(y_train)
y_val_one_hot = to_categorical(y_val)
y_test_one_hot = to_categorical(y_test)

Hyperparameter Tuning

In [15]:
# Define the objective function for Optuna
def objective(trial, X_train, y_train, X_val, y_val):
    lr = trial.suggest_loguniform('lr', 1e-5, 1e-3)
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64])

    vit_model = vit.vit_b32(
        image_size=224,
        activation='softmax',
        pretrained=True,
        include_top=False,
        pretrained_top=False,
        classes=4
    )

    model = tf.keras.Sequential([
        vit_model,
        tf.keras.layers.Flatten(),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(11, activation=tfa.activations.gelu),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(4, activation='softmax')
    ], name='vision_transformer')

    optimizer = tfa.optimizers.RectifiedAdam(learning_rate=lr)
    model.compile(optimizer=optimizer,
                  loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.2),
                  metrics=['accuracy'])

    history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=batch_size, verbose=0)
    val_loss = history.history['val_loss'][-1]

    return val_loss

In [ ]:
# Optimize hyperparameters with Optuna
study = optuna.create_study(direction='minimize')
study.optimize(lambda trial: objective(trial, X_train, y_train_one_hot, X_val, y_val_one_hot), n_trials=10)

# Get the best parameters
best_params = study.best_params
print("Best parameters:", best_params)

[I 2024-05-29 02:54:23,973] A new study created in memory with name: no-name-f18da2b5-e3ed-4b60-bc3e-786955d94045
<ipython-input-15-4f8a48eab8a4>:3: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform('lr', 1e-5, 1e-3)
[I 2024-05-29 02:59:09,549] Trial 0 finished with value: 1.0764416456222534 and parameters: {'lr': 0.00012039666724157112, 'batch_size': 16}. Best is trial 0 with value: 1.0764416456222534.
[I 2024-05-29 03:03:56,313] Trial 1 finished with value: 1.2050514221191406 and parameters: {'lr': 0.0003104371306175066, 'batch_size': 16}. Best is trial 0 with value: 1.0764416456222534.
[I 2024-05-29 03:07:42,927] Trial 2 finished with value: 0.8738086223602295 and parameters: {'lr': 0.00011705756200611183, 'batch_size': 64}. Best is trial 2 with value: 0.8738086223602295.
[I 2024-05-29 03:11:31,762]

Training & Test

In [ ]:
vit_model = vit.vit_b32(
        image_size = 224,
        activation = 'softmax',
        pretrained = True,
        include_top = False,
        pretrained_top = False,
        classes = 4)

In [ ]:
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

In [ ]:
X_train.all()

In [ ]:
#def train_vgg16_model():

  #Freeze the base model layers
  #for layer in base_model.layers:
  #    layer.trainable = False

# Add custom top layers for classification
model = tf.keras.Sequential([
        vit_model,
        tf.keras.layers.Flatten(),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(11, activation = tfa.activations.gelu),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(4, 'softmax')
    ],
    name = 'vision_transformer')

model.summary()

  # Compile model
  # optimizer = Adam(lr=5.520238899015578e-05)
learning_rate = 1e-4

optimizer = tfa.optimizers.RectifiedAdam(learning_rate=best_params['lr'])

model.compile(optimizer = optimizer,
                loss = tf.keras.losses.CategoricalCrossentropy(label_smoothing = 0.2),
                metrics = ['accuracy'])

batch_size = best_params['batch_size']

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor = 'val_accuracy',
                                                factor = 0.2,
                                                patience = 2,
                                                verbose = 1,
                                                min_delta = 1e-4,
                                                min_lr = 1e-6,
                                                mode = 'max')

earlystopping = tf.keras.callbacks.EarlyStopping(monitor = 'val_accuracy',
                                                min_delta = 1e-4,
                                                patience = 5,
                                                mode = 'max',
                                                restore_best_weights = True,
                                                verbose = 1)

checkpointer = tf.keras.callbacks.ModelCheckpoint(filepath = './model.hdf5',
                                                  monitor = 'val_accuracy',
                                                  verbose = 1,
                                                  save_best_only = True,
                                                  save_weights_only = True,
                                                  mode = 'max')

callbacks = [earlystopping, reduce_lr, checkpointer]

model.fit(x = X_train,
          y = y_train_one_hot,
          # steps_per_epoch = batch_size,
          validation_data = (X_val, y_val_one_hot),
          # validation_steps = batch_size,
          epochs = 50,
          callbacks = callbacks)


In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test_one_hot)
print('Test accuracy:', test_acc)

## GRAD-CAM

In [ ]:
cd /content/drive/MyDrive/

In [ ]:
!git clone https://github.com/jacobgil/pytorch-grad-cam.git
!cd pytorch-grad-cam
!pip install -e .

In [ ]:
cd pytorch-grad-cam

In [ ]:
!pip install -e

In [ ]:
!pip install timm

In [ ]:
pip install ttach

In [ ]:
import torch
from torch.nn import functional as F
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image, preprocess_image
from torchvision.transforms import Compose, Resize, ToTensor, Normalize
from transformers import ViTModel, ViTConfig
import timm
import numpy as np
import cv2
from PIL import Image

In [ ]:
def reshape_transform(tensor, height=14, width=14):
    result = tensor[:, 1:, :].reshape(tensor.size(0),
                                      height, width, tensor.size(2))

    # Bring the channels to the first dimension,
    # like in CNNs.
    result = result.transpose(2, 3).transpose(1, 2)
    return result

In [ ]:
def vit_grad_cam(image_path, class_idx):
    # Load ViT model
    model = torch.hub.load('facebookresearch/deit:main',
                           'deit_tiny_patch16_224', pretrained=True)
    model.eval()
    target_layers = [model.blocks[-1].norm1]

    rgb_img = cv2.imread(image_path, 1)[:, :, ::-1]
    rgb_img = cv2.resize(rgb_img, (224, 224))
    rgb_img = np.float32(rgb_img) / 255
    input_tensor = preprocess_image(rgb_img, mean=[0.5, 0.5, 0.5],
                                    std=[0.5, 0.5, 0.5])

    cam = GradCAM(model=model, target_layers=target_layers, reshape_transform=reshape_transform)
    grayscale_cam = cam(input_tensor=input_tensor)

    grayscale_cam = grayscale_cam[0, :]
    cam_image = show_cam_on_image(rgb_img, grayscale_cam)
    return cam_image

In [ ]:
cd /content/drive/My Drive/data/Final_data

In [ ]:
# Set image path and target class index
image_path = 'incipient/crop_D25_19751681-60a5-11ec-8402-0a7404972c70.png'
image_pathes = os.listdir('./incipient/')
print(image_pathes[:15])
class_idx = 0

# Get Grad-CAM image
# grad_cam_image = vit_grad_cam(image_path, class_idx)
# cv2.imwrite('grad_cam.jpg', grad_cam_image)

# Display Grad-CAM image
# img = Image.fromarray(grad_cam_image)
# display(img)

In [ ]:
fig, axes = plt.subplots(3, 5, figsize=(15, 9))

for i, ax in enumerate(axes.flatten()):
    image_path = 'incipient/' + image_pathes[i]
    class_idx = 0
    image = cv2.imread(image_path)
    ax.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
plt.tight_layout()
plt.show()


In [ ]:
fig, axes = plt.subplots(3, 5, figsize=(15, 9))

for i, ax in enumerate(axes.flatten()):
    image_path = 'incipient/' + image_pathes[i]
    class_idx = 0
    grad_cam_image = vit_grad_cam(image_path, class_idx)
    cv2.imwrite('grad_cam.jpg', grad_cam_image)
    ax.imshow(grad_cam_image)
plt.tight_layout()
plt.show()

## LIME

In [ ]:
!pip install lime

In [ ]:
from PIL import Image
import numpy as np
from skimage.segmentation import mark_boundaries
from lime import lime_image

def visualize_image(image_data):
  # LIME 설명 인스턴스 생성
  explainer = lime_image.LimeImageExplainer()

  # LIME이 필요한 이미지 하나 불러오기
  # image = incipient_data[0]
  image_15 = image_data[:15]

  fig, axes = plt.subplots(3, 5, figsize=(15, 9))

  for i, ax in enumerate(axes.flatten()):
    image = image_15[i]

    # LIME 설명 생성
    explanation = explainer.explain_instance(image, model.predict, top_labels=1, hide_color=0, num_samples=1000)

    # 설명된 이미지와 설명에 대한 마스크 가져오기
    temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=False, num_features=5, hide_rest=False)

    # 설명된 이미지와 마스크를 원본 이미지에 적용
    marked_image = mark_boundaries(temp / 2 + 0.5, mask)

    ax.imshow(marked_image)
    ax.set_title(labels[explanation.top_labels[0]])
    ax.axis('off')

  plt.tight_layout()
  plt.show()
    # # 이미지 출력
    # plt.imshow(marked_image)
    # plt.axis('off')
    # plt.show()

In [ ]:
visualize_image(incipient_data)